This colab notebook, splits the Founta, Wiki and EA classifiers to train, test and dev and saves them in the folder "Data". Then, trains a binary roberta-based classifier using the trainer module of huggingface and tests it on multiple datasets. 


In [ ]:
Dataset = 'Founta'
pre_model = 'roberta'
eval = False

In [ ]:
pip install transformers  # transformers-4.3.3

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
cd 'working Directory'
!mkdir 'Data'

In [ ]:
import re
from tqdm import tqdm
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix,precision_recall_fscore_support
import torch
import warnings
import pandas as pd
import numpy as np 
from transformers import RobertaForSequenceClassification, RobertaTokenizerFast, Trainer, TrainingArguments
import torch
warnings.filterwarnings('ignore')

In [ ]:
from word_process import Word_Preprocessing
processor = Word_Preprocessing()

#Data

In [ ]:
if Dataset =='Founta':
    processor = Word_Preprocessing()
    #relabeled by toxic-debias 
    train_ids_relabeled = pd.read_csv("Toxic_Debias/ND_founta_trn_relabel.csv")  #we use the splits of founta from https://github.com/XuhuiZhou/Toxic_Debias/tree/main/data/founta
    dev_ids_relabeled = pd.read_csv('Toxic_Debias/data/founta/dev.csv')
    test_ids_relabeled = pd.read_csv('Toxic_Debias/data/founta/test.csv')
    #original Founta
    Founta_all = pd.read_excel('Founta/hatespeech_text_label_vote.xlsx' , header = None)

    
    Founta_dev = Founta_all.iloc[dev_ids_relabeled['id'].apply(lambda x :int(x[1:])).values.tolist()]
    Founta_dev = Founta_dev.rename(columns={0:'text',1:'label'})
    Founta_dev['label'] = Founta_dev['label'].apply(lambda x:0 if x=='normal' else 1)
    Founta_dev =processor.process_all(Founta_dev,'text')
    dev_data = Founta_dev.reset_index()[['text','label']]
    dev_data.to_csv('Data/Founta_original_dev.csv')

    Founta_test = Founta_all.iloc[test_ids_relabeled['id'].apply(lambda x :int(x[1:])).values.tolist()]
    Founta_test = Founta_test.rename(columns={0:'text',1:'label'})
    Founta_test['label'] = Founta_test['label'].apply(lambda x:0 if x=='normal' else 1)
    Founta_test =processor.process_all(Founta_test,'text')
    test_data = Founta_test.reset_index()[['text','label']]
    test_data.to_csv('Data/Founta_original_test.csv')

    
    
    
    print('preparing Founta dataset ')
    Founta_train = Founta_all.iloc[train_ids_relabeled['id'].apply(lambda x :int(x[1:])).values.tolist()]
    Founta_train = Founta_train.rename(columns={0:'text',1:'label'})
    Founta_train['label'] = Founta_train['label'].apply(lambda x:0 if x=='normal' else 1)
    Founta_train =processor.process_all(Founta_train,'text')
    train_data = Founta_train.reset_index()[['text','label']]
    train_data.to_csv('Data/Founta_original_train.csv')
  


  1%|          | 68/10970 [00:00<00:33, 329.83it/s]

Start eliminate url: : )


100%|██████████| 27/27 [00:00<00:00, 253.05it/s]

Start eliminate username: : )
Start eliminate symbol: : )


Start Final check: 
finished!!


...

preparing Founta dataset 
Start eliminate url: : )


  0%|          | 0/27 [00:00<?, ?it/s]

Start eliminate username: : )
Start eliminate symbol: : )


100%|██████████| 27/27 [00:00<00:00, 41.89it/s]


Start Final check: 
finished!!


In [ ]:
# EA data read
label_dict = {'counter_speech':1,
                'discussion_of_eastasian_prejudice':1,
                'entity_directed_criticism':3,
                'entity_directed_hostility':2,
                'none_of_the_above':0}



EA_data = pd.read_csv('EA/hs_AsianPrejudice_20kdataset_cleaned_anonymized.tsv',sep='\t',encoding = "ISO-8859-1")


EA_data = processor.process_all(EA_data,'text')
EA_data['label'] = EA_data['expert'].apply(lambda x:label_dict[x])
EA_data = EA_data[['text','label']]

train_size = 0.8
dev_size = 0.3

EA_train_stratified=EA_data.groupby('label').apply(lambda x: x.sample(frac=train_size, random_state= 100))  #stratified sampling 
EA_train = EA_data.iloc[[item[1] for item in EA_train_stratified.index]]
EA_test=EA_data.drop(EA_train.index).reset_index(drop=True)

EA_dev_stratified=EA_test.groupby('label').apply(lambda x: x.sample(frac=dev_size, random_state= 100))
EA_dev = EA_test.iloc[[item[1] for item in EA_dev_stratified.index]]
EA_test=EA_test.drop(EA_dev.index).reset_index(drop=True)

EA_train = EA_train.reset_index(drop=True)
EA_dev = EA_dev.reset_index(drop=True)

train_data =EA_train 
dev_data = EA_dev
test_data = EA_test

EA_train.to_csv('Data/EA_train.csv')
EA_dev.to_csv('Data/EA_dev.csv')
EA_test.to_csv('Data/EA_test.csv')


  0%|          | 41/20000 [00:00<00:50, 394.16it/s]

Start eliminate url: : )


  0%|          | 15/20000 [00:00<02:13, 149.92it/s]

Start eliminate username: : )


 63%|██████▎   | 17/27 [00:00<00:00, 163.92it/s]

Start eliminate symbol: : )


100%|██████████| 27/27 [00:00<00:00, 125.62it/s]


Start Final check: 
finished!!


In [ ]:
if Dataset=='Toxic':
  data = pd.read_csv('wiki/wiki_lda_topics_lda_probabilities.csv')  #from https://arxiv.org/abs/2010.07414
  data['toxicity'] = data['toxicity'].apply(lambda x:1 if x else 0)
  data = data.rename(columns={'comment':'text','toxicity':'label'})

  topic_categories={1:[0,1],
                  2:[2,7,8,9,12,14,16],
                  3:[3,4,5,6,10,11,13,15,17,18,19]}

  
  if Dataset == 'Toxic':
    toxic_train = data[data['split']=='train' ][data['wiki_topic'].isin(topic_categories[1]+topic_categories[2])][['text','label']]
  if Dataset =='Toxic-original':
    toxic_train = data[data['split']=='train' ][['text','label']]
  toxic_test = data[data['split']=='test'][['text','label']]
  toxic_dev = data[data['split']=='dev'][['text','label']]

  train_data =toxic_train 
  dev_data = toxic_dev
  test_data = toxic_test

  train_data.to_csv('Data/wiki_train.csv')
  dev_data.to_csv('Data/wiki_dev.csv')
  test_data.to_csv('Data/Wiki_test.csv')

In [ ]:
if torch.cuda.is_available():
    device = torch.device('cuda:0')
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
else:
    device = torch.device('cpu')

print(device)

cuda:0


In [ ]:

if eval:
  model = RobertaForSequenceClassification.from_pretrained('models/exp-Toxicity-roberta')#trainer.model
  tokenizer = RobertaTokenizerFast.from_pretrained('models/exp-Toxicity-roberta')

else:

  if pre_model == 'roberta':
    model = RobertaForSequenceClassification.from_pretrained('roberta-base',num_labels = len(train_data['label'].unique()))
    tokenizer = RobertaTokenizerFast.from_pretrained('roberta-base')

  if pre_model == 'bert':
    model = BertForSequenceClassification.from_pretrained('bert-base-uncased',num_labels = len(train_data['label'].unique()))
    tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')


Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.weight', 'classifie

In [ ]:
model.num_labels 

2

In [ ]:
model

In [ ]:
class ToxicityDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

In [ ]:
train_encodings = tokenizer(train_data['text'].values.tolist(), truncation=True, padding=True)
val_encodings = tokenizer(dev_data['text'].values.tolist(), truncation=True, padding=True)

train_labels =train_data['label'].values.tolist()
val_labels = dev_data['label'].values.tolist()

train_dataset = ToxicityDataset(train_encodings, train_labels)
val_dataset = ToxicityDataset(val_encodings, val_labels)


In [ ]:
test_encodings = tokenizer(test_data['text'].values.tolist(), truncation=True, padding=True)
test_labels = test_data['label'].values.tolist()
test_dataset = ToxicityDataset(test_encodings, test_labels)

In [ ]:
def compute_metrics(pred, average = 'macro'):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average=average)
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

In [ ]:
if pre_model=='roberta':
  train_epoch = 3
if pre_model == 'bert':
  train_epoch = 3

training_args = TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=train_epoch,              # total # of training epochs
    per_device_train_batch_size=16,  # batch size per device during training
    per_device_eval_batch_size=64,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
)

trainer = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=val_dataset            # evaluation dataset
)

In [ ]:
trainer.train()

Step,Training Loss
500,0.324600
1000,0.235800
1500,0.233400
2000,0.216000
2500,0.207100
3000,0.226500
3500,0.223200
4000,0.218100
4500,0.232000
5000,0.210400


TrainOutput(global_step=11646, training_loss=0.21724679653083215, metrics={'train_runtime': 4853.9493, 'train_samples_per_second': 2.399, 'total_flos': 58243007105589240, 'epoch': 3.0})

In [ ]:
preds = trainer.predict(test_dataset)

In [ ]:
compute_metrics(preds,average = 'macro')

In [ ]:
compute_metrics(preds,average = 'micro')

In [ ]:
trainer.save_model('models/exp-Toxicity-roberta')
tokenizer.save_pretrained('models/exp-Toxicity-roberta')